<a href="https://colab.research.google.com/github/shykiu77/PLN_QA_LLM_2025_Melo_Carlos/blob/main/PLN_QA_LLM_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Instalação de arquivos necessários
!pip install python-docx -q
!pip install pymupdf -q
!pip install sentence-transformers -q
!pip install faiss-cpu -q
!pip install transformers -q
!pip install accelerate  -q
!pip install bitsandbytes -q
!pip install pandas -q
!pip install matplotlib -q
!pip install langchain -q


!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
!pip install huggingface_hub 
!pip install ipywidgets

Looking in indexes: https://download.pytorch.org/whl/cu126


In [10]:
# Importação de bibliotecas
# from google.colab import files
from docx import Document
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import matplotlib.pyplot as plt
import fitz
import re
import faiss

In [ ]:
# Upload manual dos arquivos necessários (encontrados na pasta 'content')
uploaded = files.upload()

In [2]:
# Funções para extrair conteúdo dos arquivos

# Extração de texto do .docx
def extract_text_from_docx(file_path):
  doc = Document(file_path)
  texts = []

  # Extrai textos fora das tabelas
  for p in doc.paragraphs:
    text = p.text.strip()
    if text:
      texts.append(text)

  # Extrai texto das tabelas
  for table in doc.tables:
    for row in table.rows:
      row_text = []
      for cell in row.cells:
        cell_text = cell.text.strip()
        if cell_text:
          row_text.append(cell_text)
      if row_text:
        texts.append(' | '.join(row_text))

  return '\n'.join(texts)


# Extração de texto do .pdf
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    full_text = []

    for page in doc:
        text = page.get_text("text")
        # Limpeza leve: remove espaços múltiplos, quebra em parágrafos
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\s([.,;:])', r'\1', text)  # remove espaço antes de pontuação
        full_text.append(text.strip())

    return "\n".join(full_text)

In [3]:
# Função para dividir em blocos menores
def split_context(text, max_len=1500):
    sentences = text.split('. ')
    blocks, current = [], ""
    for s in sentences:
        if len(current + s) <= max_len:
            current += s + ". "
        else:
            blocks.append(current.strip())
            current = s + ". "
    if current:
        blocks.append(current.strip())
    return blocks


In [4]:
# Extração dos documentos
docx_text = extract_text_from_docx('./content/DICIONARIO_DE_DADOS.docx')
pdf_text = extract_text_from_pdf('./content/doencas_respiratorias_cronicas.pdf')

In [ ]:
# Aplicação dos modelos nas perguntas

# Modelos de teste
model_names = [
    "deepset/roberta-base-squad2",
    "distilbert-base-cased-distilled-squad",
    "timpal0l/mdeberta-v3-base-squad2"
]

documentos = {
    "Dicionário de Dados": docx_text,
    "Doenças Respiratórias Crônicas": pdf_text
}

# Perguntas e respostas para cada documento
perguntas_respostas_por_doc = {
    "Dicionário de Dados": {
        "Qual o nome da tabela LFCES004 no banco de produção federal?": "TB_ESTABELECIMENTO",
        "Na tabela RL_ESTAB_COMPLEMENTAR, o que representa o campo QTDE_SUS?": "Quantidade de Leitos Disponíveis para o SUS.( Para os Estabelecimentos vinculados ao SUS.)",
        "Quais são os domínios do campo INDGESTOR, da tabela FCESGEST?": "1 – Secretaria Estadual, 2 – Região de Saúde, 3 – Secretaria Municipal Não Pleno, 4 – Módulo Assistencial, 5 – Distrito Sanitário, 6 – DataSus, 7 – 5éc. Municipal Pleno, 8 – Capital, 9 – Microrregião"
    },
    "Doenças Respiratórias Crônicas": {
        "Quais são os principais fatores de risco preveníveis para doenças respiratórias crônicas (DRC)?": "Tabagismo, poluição ambiental, alérgenos, agentes ocupacionais, pneumonia, bronquiolite e tuberculose.",
        "Qual é a definição clássica de sintomático respiratório?": "É o indivíduo que apresenta tosse, associada ou não a outra alteração respiratória, por algumas semanas.",
        "Como a tosse pode ser classificada?": "Em aguda até três semanas de duração; subaguda de três a oito semanas; e crônica se superior a oito semanas."
    }
}

resultados = []

for model_name in model_names:
    print(f"Rodando Modelo: {model_name}")
    qa = pipeline("question-answering", model=model_name, tokenizer=model_name, device=0)

    for nome_doc, perguntas_respostas in perguntas_respostas_por_doc.items():
        contexto = documentos[nome_doc]
        blocos = split_context(contexto, 1500)

        for pergunta, resposta_correta in perguntas_respostas.items():
            print(f"Rodando Pergunta: {pergunta}")
            melhor_resposta = {"answer": "", "score": 0.0}
            for bloco in blocos:
              if not bloco.strip():
                continue
              try:
                resultado = qa(question=pergunta, context=bloco)
                if resultado["score"] > melhor_resposta["score"]:
                  melhor_resposta = resultado
              except Exception as e:
                print(f"Erro ao processar bloco: {e}")
                continue

            resposta_modelo = melhor_resposta["answer"]
            score = melhor_resposta["score"]

            resultados.append({
                "Modelo": model_name,
                "Documento": nome_doc,
                "Pergunta": pergunta,
                "Resposta": resposta_modelo,
                "Resposta Esperada": resposta_correta,
                "Confiança": round(score, 4)
            })

In [ ]:
# Avaliação manual + Similaridade semântica

# Avaliação manual
def avaliar_resposta(r_modelo, r_correta):
    r_modelo = r_modelo.lower().strip()
    r_correta = r_correta.lower().strip()
    if r_modelo == r_correta:
        return "✔️"
    elif r_modelo in r_correta or r_correta in r_modelo:
        return "➖"
    else:
        return "❌"

# Carrega modelo para embeddings
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Tabela com avaliações
avaliacoes = []

for r in resultados:
    r_modelo = r["Resposta"]
    r_correta = r["Resposta Esperada"]

    # Similaridade
    emb1 = embedding_model.encode(r_modelo, convert_to_tensor=True)
    emb2 = embedding_model.encode(r_correta, convert_to_tensor=True)
    similaridade = util.cos_sim(emb1, emb2).item()

    # Avaliação manual
    avaliacao = avaliar_resposta(r_modelo, r_correta)

    avaliacoes.append({
        "Documento": r["Documento"],
        "Pergunta": r["Pergunta"],
        "Modelo": r["Modelo"],
        "Resposta": r_modelo,
        "Resposta Esperada": r_correta,
        "Avaliação": avaliacao,
        "Similaridade": round(similaridade, 4),
        "Confiança": r["Confiança"]
    })

In [ ]:
# Exibir tabela de avaliação

def exibir_resultados_formatados(df):
    modelos = df["Modelo"].unique()

    for modelo in modelos:
        print(f"\n\nResultados para: {modelo}")
        df_modelo = df[df["Modelo"] == modelo]
        styled = df_modelo.style.set_properties(**{'text-align': 'center'}) \
            .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) \
            .set_caption(f"Modelo: {modelo}")
        display(styled)

    print("\n\n\n")
    # Gráfico comparativo de avaliações
    comparativo = df.groupby(["Modelo", "Avaliação"]).size().unstack(fill_value=0)

    # Ordena colunas para manter ✔️ ➖ ❌ nessa ordem
    colunas_ordenadas = ["✔️", "➖", "❌"]
    comparativo = comparativo[[col for col in colunas_ordenadas if col in comparativo.columns]]

    # Define cores personalizadas
    cores = {
        "✔️": "green",
        "➖": "orange",
        "❌": "red"
    }
    cores_usadas = [cores[c] for c in comparativo.columns]

    # Plota
    ax = comparativo.plot(
        kind="bar",
        stacked=True,
        figsize=(10, 6),
        color=cores_usadas,
        edgecolor='black'
    )

    plt.title("Avaliação das Respostas por Modelo", fontsize=14)
    plt.xlabel("Modelo", fontsize=12)
    plt.ylabel("Quantidade de Respostas", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Legenda personalizada
    legenda = {
        "✔️": "Correta",
        "➖": "Parcialmente Correta",
        "❌": "Incorreta"
    }
    handles, labels = ax.get_legend_handles_labels()
    new_labels = [legenda.get(lbl, lbl) for lbl in labels]
    ax.legend(
    handles, new_labels,
    title="Avaliação",
    loc='center left',
    bbox_to_anchor=(1, 0.5)
    )

    plt.tight_layout()
    plt.show()


df_avaliacao = pd.DataFrame(avaliacoes)
exibir_resultados_formatados(df_avaliacao)

In [6]:
# Modelo de embedding multilíngue
print("Carregando modelo de embedding 'paraphrase-multilingual-mpnet-base-v2'...")
embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Função de divisão de texto com sobreposição
def split_context_com_overlap(text, max_len=512,overlap=32):
    sentences = re.split(r'(?<=[.!?])\s+', text.replace('\n', ' '))
    blocks = []
    current_block = ""
    for sentence in sentences:
        if len(current_block) + len(sentence) + 1 <= max_len:
            current_block += sentence + " "
        else:
            if current_block: blocks.append(current_block.strip())
            start_index = max(0, len(current_block) - overlap)
            current_block = current_block[start_index:] + " " + sentence + " "
    if current_block: blocks.append(current_block.strip())
    return blocks

# Combinar todos os textos e criar os blocos
texto_completo = docx_text + "\n\n" + pdf_text
blocos_de_texto = split_context_com_overlap(texto_completo)
print(f"Texto dividido em {len(blocos_de_texto)} blocos para indexação.")

# Criar os embeddings
print("Criando embeddings para os blocos de texto...")
embeddings = embedding_model.encode(blocos_de_texto, convert_to_tensor=True, show_progress_bar=True)

# Normalizar e criar índice FAISS
embeddings_cpu = embeddings.cpu().numpy()
faiss.normalize_L2(embeddings_cpu)
embedding_dim = embeddings_cpu.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(embeddings_cpu)
print("Índice FAISS para busca de similaridade criado com sucesso.")

Carregando modelo de embedding 'paraphrase-multilingual-mpnet-base-v2'...
Texto dividido em 765 blocos para indexação.
Criando embeddings para os blocos de texto...


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Índice FAISS para busca de similaridade criado com sucesso.


In [ ]:
def get_rag_answer_with_context(question, qa_pipeline, top_k=5):
    question_embedding = embedding_model.encode([question]).astype('float32')
    faiss.normalize_L2(question_embedding)
    _, indices = index.search(question_embedding, top_k)
    relevant_contexts = [blocos_de_texto[i] for i in indices[0]]

    best_answer = {"answer": "", "score": 0.0}
    best_context = ""

    for context in relevant_contexts:
        if not context.strip(): continue
        result = qa_pipeline(question=question, context=context)
        if result["score"] > best_answer["score"]:
            best_answer = result
            best_context = context

    return best_answer, best_context,relevant_contexts

qa_model_names = [
    "deepset/roberta-base-squad2",
    "distilbert-base-cased-distilled-squad",
    "timpal0l/mdeberta-v3-base-squad2"
]

perguntas_respostas_por_doc = {
     "Dicionário de Dados": {
        "Qual o nome da tabela LFCES004 no banco de produção federal?": "TB_ESTABELECIMENTO",
        "Na tabela RL_ESTAB_COMPLEMENTAR, o que representa o campo QTDE_SUS?": "Quantidade de Leitos Disponíveis para o SUS.( Para os Estabelecimentos vinculados ao SUS.)",
        "Quais são os domínios do campo INDGESTOR, da tabela FCESGEST?": "1 – Secretaria Estadual, 2 – Região de Saúde, 3 – Secretaria Municipal Não Pleno, 4 – Módulo Assistencial, 5 – Distrito Sanitário, 6 – DataSus, 7 – 5éc. Municipal Pleno, 8 – Capital, 9 – Microrregião"
    },
    "Doenças Respiratórias Crônicas": {
        "Quais são os principais fatores de risco preveníveis para doenças respiratórias crônicas (DRC)?": "Tabagismo, poluição ambiental, alérgenos, agentes ocupacionais, pneumonia, bronquiolite e tuberculose.",
        "Qual é a definição clássica de sintomático respiratório?": "É o indivíduo que apresenta tosse, associada ou não a outra alteração respiratória, por algumas semanas.",
        "Como a tosse pode ser classificada?": "Em aguda até três semanas de duração; subaguda de três a oito semanas; e crônica se superior a oito semanas."
    }
}

final_results = []
print("\n--- Iniciando Avaliação e Captura de Contexto ---")

for model_name in qa_model_names:
    print(f"\nCarregando e avaliando modelo: {model_name}")
    qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

    for doc_name, qa_pairs in perguntas_respostas_por_doc.items():
        for question, correct_answer in qa_pairs.items():
            print(f"  -> Pergunta: {question[:60]}...")
            rag_result, used_context, all_context = get_rag_answer_with_context(question, qa_pipeline)
            final_results.append({
                "Modelo": model_name.split('/')[-1],
                "Documento": doc_name,
                "Pergunta": question,
                "Resposta": rag_result["answer"],
                "Resposta Esperada": correct_answer,
                "Confiança": round(rag_result["score"], 4),
                "Contexto Utilizado": used_context,
                "Todo Contexto": all_context
            })

print("\n--- Avaliação concluída! ---")

In [ ]:
def evaluate_manual_answer(model_ans, correct_ans):
    model_ans_lower = model_ans.lower().strip().replace('.', '')
    correct_ans_lower = correct_ans.lower().strip().replace('.', '')
    if not model_ans_lower: return "❌"
    if model_ans_lower == correct_ans_lower: return "✔️"
    if model_ans_lower in correct_ans_lower or correct_ans_lower in model_ans_lower: return "➖"
    return "❌"

evaluations = []
for r in final_results:
    model_answer = r["Resposta"]
    correct_answer = r["Resposta Esperada"]

    emb1 = embedding_model.encode(model_answer, convert_to_tensor=True)
    emb2 = embedding_model.encode(correct_answer, convert_to_tensor=True)
    similarity = util.cos_sim(emb1, emb2).item()
    manual_eval = evaluate_manual_answer(model_answer, correct_answer)

    evaluations.append({
        "Modelo": r["Modelo"],
        "Documento": r["Documento"],
        "Pergunta": r["Pergunta"],
        "Resposta": model_answer,
        "Resposta Esperada": correct_answer,
        "Avaliação": manual_eval,
        "Similaridade": round(similarity, 4),
        "Confiança": r["Confiança"]
    })

df_evaluation = pd.DataFrame(evaluations)

print("\n--- Tabela Comparativa de Desempenho dos Modelos com RAG Melhorada ---")
display(df_evaluation.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])]))

print("\n--- Gráfico Comparativo de Avaliações por Modelo ---")
comparativo = df_evaluation.groupby(["Modelo", "Avaliação"]).size().unstack(fill_value=0)
colunas_ordenadas = [col for col in ["✔️", "➖", "❌"] if col in comparativo.columns]
comparativo = comparativo[colunas_ordenadas]

ax = comparativo.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 7),
    color={"✔️": "green", "➖": "orange", "❌": "red"},
    edgecolor='black'
)

plt.title("Desempenho dos Modelos de Q&A com Abordagem RAG Melhorada", fontsize=16)
plt.xlabel("Modelo", fontsize=12)
plt.ylabel("Quantidade de Respostas", fontsize=12)
plt.xticks(rotation=0, ha='center')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title="Avaliação", labels=["Correta", "Parcialmente Correta", "Incorreta"])
plt.tight_layout()
plt.show()

In [ ]:
for result in final_results:
    print("-" * 80)
    print(f"Modelo: {result['Modelo']}")
    print(f"Pergunta: {result['Pergunta']}")
    print(f"Resposta Gerada: {result['Resposta']} (Confiança: {result['Confiança']})")
    print("\n--- Contexto Completo ---")
    print(result['Todo Contexto'])
    print("\n--- Contexto Utilizado para a Resposta ---")
    print(result['Contexto Utilizado'])
    print("-" * 80 + "\n")

---------------------------------

In [9]:

import faiss
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- 1. Carregar os modelos ---
print("Carregando modelo de embedding (para busca inicial)...")
# Bi-Encoder: Rápido, para buscar um grande número de candidatos
embedding_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

print("Carregando modelo de Cross-Encoder (para re-ranking de precisão)...")
# Cross-Encoder: Lento mas preciso, para reclassificar os melhores candidatos
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


# --- 2. Dividir o Texto (Chunking Inteligente) ---
# Usamos o RecursiveCharacterTextSplitter, que é mais robusto
# Ele tenta dividir por parágrafos, depois por linhas, depois por frases.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # Aumenta o tamanho do chunk para manter o contexto
    chunk_overlap=50, # Sobreposição para não perder informações nas bordas
    length_function=len,
    is_separator_regex=False,
)

# Supondo que você já tenha 'texto_completo'
texto_completo = docx_text + "\n\n" + pdf_text
blocos_de_texto = text_splitter.split_text(texto_completo)
print(f"Texto dividido em {len(blocos_de_texto)} blocos com a nova estratégia.")


# --- 3. Criar Embeddings e Índice FAISS (Como antes) ---
print("Criando embeddings para os blocos de texto...")
embeddings = embedding_model.encode(blocos_de_texto, convert_to_tensor=True, show_progress_bar=True)
embeddings_cpu = embeddings.cpu().numpy()

# Normalizar para usar o produto interno (similaridade de cosseno)
faiss.normalize_L2(embeddings_cpu)

embedding_dim = embeddings_cpu.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(embeddings_cpu)
print("Índice FAISS para busca de similaridade criado com sucesso.")

Carregando modelo de embedding (para busca inicial)...
Carregando modelo de Cross-Encoder (para re-ranking de precisão)...
Texto dividido em 1160 blocos com a nova estratégia.
Criando embeddings para os blocos de texto...


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

Índice FAISS para busca de similaridade criado com sucesso.


In [11]:
class RAGMelhorado:
    def __init__(self, embedding_model, cross_encoder, faiss_index, text_chunks, generation_pipeline, tokenizer):
        self.embedding_model = embedding_model
        self.cross_encoder = cross_encoder
        self.index = faiss_index
        self.text_chunks = text_chunks
        self.generation_pipeline = generation_pipeline
        self.tokenizer = tokenizer

    def _recuperar_documentos(self, question, top_k_initial=10):
        """Etapa 1: Recupera os top_k_initial documentos mais prováveis usando FAISS."""
        question_embedding = self.embedding_model.encode([question]).astype('float32')
        faiss.normalize_L2(question_embedding)
        _, indices = self.index.search(question_embedding, top_k_initial)
        return [self.text_chunks[i] for i in indices[0]]

    def _reclassificar_documentos(self, question, documents, top_k_final=3):
        """Etapa 2: Reclassifica os documentos recuperados usando o Cross-Encoder."""
        # O Cross-Encoder precisa de pares [pergunta, documento]
        pairs = [[question, doc] for doc in documents]
        scores = self.cross_encoder.predict(pairs)
        
        # Combina documentos e scores e ordena pelo score
        doc_scores = list(zip(documents, scores))
        doc_scores.sort(key=lambda x: x[1], reverse=True)
        
        # Retorna os top_k_final melhores documentos
        return [doc for doc, score in doc_scores[:top_k_final]]

    def responder(self, question, max_context_tokens=3000):
        """Executa o pipeline completo de RAG: Recuperar, Reclassificar e Gerar."""
        # Etapa 1: Recuperação inicial rápida
        documentos_iniciais = self._recuperar_documentos(question)
        
        # Etapa 2: Reclassificação de precisão
        documentos_reclassificados = self._reclassificar_documentos(question, documentos_iniciais)
        
        # Etapa 3: Construção do Contexto e Geração (como na sua função aprimorada)
        contexto_utilizado = ""
        current_tokens = 0
        for text_block in documentos_reclassificados:
            block_tokens = len(self.tokenizer.encode(text_block))
            if current_tokens + block_tokens <= max_context_tokens:
                contexto_utilizado += text_block + "\n\n"
                current_tokens += block_tokens
            else:
                break
        
        contexto_utilizado = contexto_utilizado.strip()

        prompt = f"""Você é um assistente de IA factual. Responda a pergunta estritamente com base no contexto fornecido. Se a resposta não estiver no contexto, diga "A informação não foi encontrada no contexto fornecido.".

Contexto:
{contexto_utilizado}

Pergunta: {question}

Resposta:
"""
        
        print(f" -> Contexto final com {current_tokens} tokens, baseado nos melhores {len(documentos_reclassificados)} chunks reclassificados.")

        with torch.no_grad():
            generated_output = self.generation_pipeline(prompt, pad_token_id=self.tokenizer.eos_token_id)
        
        resposta_completa = generated_output[0]["generated_text"]
        resposta_gerada = resposta_completa[len(prompt):]

        return resposta_gerada, contexto_utilizado, documentos_reclassificados

In [14]:

llama_model_name = "google/gemma-2b-it"

# Configuração para carregar o modelo em 8-bit e economizar memória
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Carregando o tokenizer, que traduz texto para tokens que o modelo entende
tokenizer = AutoTokenizer.from_pretrained(llama_model_name)

# Carregando o modelo principal
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Coloca o modelo automaticamente na GPU se disponível
)

# Criando o pipeline, que é uma interface de alto nível para usar o modelo
llama_pipeline = pipeline(
    "text-generation",
    model=llama_model,
    tokenizer=tokenizer,
    max_new_tokens=512  # Limita o tamanho da resposta
)

sistema_rag = RAGMelhorado(
    embedding_model=embedding_model,
    cross_encoder=cross_encoder,
    faiss_index=index,
    text_chunks=blocos_de_texto,
    generation_pipeline=llama_pipeline,
    tokenizer=tokenizer
)

# --- Loop de Avaliação ---
print("\n--- Iniciando avaliação com o sistema RAG APRIMORADO ---")
final_results = []

for doc_name, qa_pairs in perguntas_respostas_por_doc.items():
    print(f"\nProcessando documento: {doc_name}")
    for question, correct_answer in qa_pairs.items():
        print(f"  -> Pergunta: {question[:80]}...")
        try:
            # A chamada agora é muito mais simples
            resposta, used_context, all_context = sistema_rag.responder(question)
            
            final_results.append({
                "Modelo": 'google/gemma-2b-it',
                "Documento": doc_name,
                "Pergunta": question,
                "Resposta": resposta.strip(),
                "Resposta Esperada": correct_answer,
                "Confiança": None,
                "Contexto Utilizado": used_context,
                "Todo Contexto": all_context
            })

        except Exception as e:
            print(f"    [ERRO] Ocorreu um erro ao processar a pergunta: {e}")
            torch.cuda.empty_cache() 
        
        torch.cuda.empty_cache()

print("\n--- Avaliação aprimorada concluída! ---")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



--- Iniciando avaliação com o sistema RAG APRIMORADO ---

Processando documento: Dicionário de Dados
  -> Pergunta: Qual o nome da tabela LFCES004 no banco de produção federal?...
 -> Contexto final com 479 tokens, baseado nos melhores 3 chunks reclassificados.
  -> Pergunta: Na tabela RL_ESTAB_COMPLEMENTAR, o que representa o campo QTDE_SUS?...
 -> Contexto final com 517 tokens, baseado nos melhores 3 chunks reclassificados.
  -> Pergunta: Quais são os domínios do campo INDGESTOR, da tabela FCESGEST?...
 -> Contexto final com 554 tokens, baseado nos melhores 3 chunks reclassificados.

Processando documento: Doenças Respiratórias Crônicas
  -> Pergunta: Quais são os principais fatores de risco preveníveis para doenças respiratórias ...
 -> Contexto final com 384 tokens, baseado nos melhores 3 chunks reclassificados.
  -> Pergunta: Qual é a definição clássica de sintomático respiratório?...
 -> Contexto final com 374 tokens, baseado nos melhores 3 chunks reclassificados.
  -> Pergunta:

In [15]:
for result in final_results:
    print("-" * 80)
    print(f"Documento: {result['Documento']}")
    print(f"Pergunta: {result['Pergunta']}")
    print(f"\nResposta Gerada:\n{result['Resposta']}")
    print(f"\nResposta Esperada:\n{result['Resposta Esperada']}")
    print("\n--- Contexto Utilizado para a Resposta ---")
    print(result['Contexto Utilizado'])
    print("-" * 80 + "\n")

--------------------------------------------------------------------------------
Documento: Dicionário de Dados
Pergunta: Qual o nome da tabela LFCES004 no banco de produção federal?

Resposta Gerada:
TB_ESTABELECIMENTO

Resposta Esperada:
TB_ESTABELECIMENTO

--- Contexto Utilizado para a Resposta ---
NOME DA TABELA (BANCO LOCAL) | NOME DA TABELA (BANCO DE PRODUÇÃO FEDERAL) | DESCRIÇÃO DA TABELA
ACESSOSUSUARIO | - | ACESSOS DOS USUÁRIOS
FCESGEST | - | PERFIL DO GESTOR
LFCES002 | RL_ESTAB_COMPLEMENTAR | LEITOS HOSPITALARES
LFCES004 | TB_ESTABELECIMENTO | ESTABELECIMENTOS DE SAÚDE
LFCES005 | TB_MANTENEDORA | MANTENEDORA
LFCES006 | RL_ESTAB_ATEND_PREST_CONV | ATENDIMENTO PRESTADO P/ UNIDADE
LFCES007 | RL_ESTAB_PROG_FUNDO | GESTÃO DE ATIVIDADES / NÍVEL DE ATENÇÃO

DT_ATUALIZACAO_ORIGEM | DATE | NULL | Data da primeira entrada no Banco de Produção Federal
DT_CMTP_INICIO | DATE | NULL | Data da Primeira entrada ou Data do Retorno no Banco de Produção Federal
DT_CMTP_FIM | DATE | NULL | Data 